📘 1️⃣ Project Introduction

# 🛡️ SecureCodeGen AI — AI-powered Code Generation & Vulnerability Checker

SecureCodeGen AI is a production-grade system that combines:

- CodeLlama-7B-Instruct (4-bit optimized)
- Secure Python code generation
- Vulnerability detection & security auditing
- Natural-language prompt → Clean Python code output
- File-based and text-based code analysis
- Automatic extraction of valid code from model outputs
- Flask web application with modern UI
- Upload system for .py files
- Structured vulnerability reports (Critical / Warnings / Good Practices)

This notebook installs the environment, explains model usage, handles security, 
and launches a fully functional web UI using ngrok.


📘 2️⃣ Install Dependencies

This step installs:

- Flask → Web framework
- pyngrok → Public URL for deployment
- transformers → Loads CodeLlama
- accelerate → Optimized inference
- bitsandbytes → 4-bit quantization for low VRAM usage
- sentencepiece → Tokenization backend
- protobuf → HF model compatibility
- Folder creation for templates, static & uploads

No modification needed here.


In [ ]:
# ===============================
# 1️⃣ Install Dependencies
# ===============================
!pip install -q flask pyngrok transformers accelerate bitsandbytes sentencepiece protobuf

In [ ]:
# ===============================
# 2️⃣ Create Directories
# ===============================
!mkdir -p templates static uploads

📘 3️⃣ Hugging Face Login (Token Removed)

Your notebook originally contained a HuggingFace token.
It has been removed for security.

To generate your own token:
1. Visit https://huggingface.co/settings/tokens
2. Click “New Token”
3. Set permission → Read
4. Copy your token
5. Add it in the notebook:

login(token="YOUR_TOKEN_HERE")

⚠️ Never upload your token to GitHub or share it publicly.


In [ ]:
# ===============================
# 3️⃣ HuggingFace Login
# ===============================
from huggingface_hub import login
login(token="YOUR_TOKEN_HERE")

📘 4️⃣ Flask App Overview

The backend implements:

### ✔ Code Generation
- Takes user description or uploaded Python file
- Builds structured prompt for CodeLlama
- Ensures output is ONLY Python code
- Extracts clean executable code from messy model output
- Enforces no comments, no Markdown — pure Python

### ✔ Vulnerability Checker
- Takes pasted code or uploaded file
- Uses cybersecurity-oriented prompt
- Generates human-readable security report:
  - 🔴 Critical issues
  - 🟡 Warnings
  - 🟢 Good practices
  - 💡 Recommendations

### ✔ Model Loading
- Uses 4-bit quantization (BitsAndBytes)
- Avoids repeated model loading using cache
- Auto GPU/CPU support

### ✔ File Handling
- Uploads stored in /uploads
- 16 MB file limit
- Safe reading of files

### ✔ HTML Rendering
- index.html → Landing page  
- generate.html → Code generator  
- check.html → Vulnerability analyzer  

All stored inside /templates.


📘 5️⃣ CodeLlama Model Details

Model: CodeLlama-7B-Instruct (HuggingFace)

Optimized using:
- 4-bit NF4 quantization
- Double quantization
- float16 compute

Benefits:
- Faster inference
- Fit into Kaggle T4 GPU easily
- High-quality code generation

Prompting Style:
- [INST] ... [/INST] style instructions
- Forces model to output only Python code
- Ensures consistent vulnerability analysis


📘 6️⃣ Code Extraction Logic

SecureCodeGen implements a multi-pass code extractor:

1. Remove [INST] blocks
2. Look for ```python ... ``` fenced code
3. Look for ``` ... ``` fallback code
4. Pattern-match Python keywords:
   - def
   - import
   - class
   - main()
5. Remove stray text or model artifacts

This ensures valid, runnable Python code is extracted,
even if the model outputs mixed text.


📘 7️⃣ Vulnerability Checker Prompt Design

The vulnerability prompt is engineered to produce:

- Clear, structured security analysis  
- No hallucinated remediation code  
- Breakdown into:
  - 🔴 Critical Issues
  - 🟡 Warnings
  - 🟢 Good Practices
  - 💡 Recommendations

Security themes covered:
- Input validation
- Injection risk
- Hardcoded credentials
- Dangerous functions
- File handling safety
- Exception handling practices


In [ ]:
# ===============================
# 4️⃣ Main Flask App (FIXED VERSION)
# ===============================
%%writefile app.py
from flask import Flask, render_template, request, redirect, url_for, flash
import functools, os, torch, re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

app = Flask(__name__)
app.secret_key = "secure_code_gen_2024"
UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

# Global variable to store model
MODEL_CACHE = {"model": None, "tokenizer": None}

def load_model():
    """Load model once and cache it"""
    if MODEL_CACHE["model"] is None:
        print("🔄 Loading CodeLlama-7B model...")
        MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"

        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
        )

        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.float16,
        )

        MODEL_CACHE["model"] = model
        MODEL_CACHE["tokenizer"] = tokenizer
        print("✅ Model loaded successfully!")

    return MODEL_CACHE["model"], MODEL_CACHE["tokenizer"]

def prompt_code_generation(task_desc):
    """Improved prompt for code generation"""
    return f"""[INST] You are an expert Python programmer. Write clean, correct, and efficient code.

Task: {task_desc}

Requirements:
- Syntactically correct Python code
- Logically correct implementation
- Include main() function if needed
- Use clear variable names
- NO comments, explanations, or markdown - ONLY code

Output the complete Python code:
[/INST]

```python
"""

def prompt_vuln_check(code):
    """Enhanced prompt for vulnerability checking"""
    return f"""[INST] You are a cybersecurity expert. Analyze this Python code for vulnerabilities.

Code:
```python
{code}
```

Provide a security report with:
🔴 Critical Issues
🟡 Warnings
🟢 Good Practices
💡 Recommendations

Be specific and actionable.
[/INST]

"""

def extract_code(response):
    """Extract clean Python code from model output"""
    try:
        # Remove the instruction prompt if present
        response = re.sub(r'\[INST\].*?\[/INST\]', '', response, flags=re.DOTALL)

        # Try to find code between ```python and ```
        match = re.search(r'```python\s*(.*?)\s*```', response, re.DOTALL)
        if match:
            code = match.group(1).strip()
            if code:
                return code

        # Try to find code between ``` and ```
        match = re.search(r'```\s*(.*?)\s*```', response, re.DOTALL)
        if match:
            code = match.group(1).strip()
            if code and ('def ' in code or 'import ' in code or 'class ' in code):
                return code

        # Look for Python code patterns
        lines = response.split('\n')
        code_lines = []
        started = False

        for line in lines:
            stripped = line.strip()
            # Start capturing when we see Python keywords
            if any(kw in stripped for kw in ['def ', 'import ', 'from ', 'class ', 'if __name__']):
                started = True

            if started:
                code_lines.append(line)

        if code_lines:
            code = '\n'.join(code_lines).strip()
            # Remove any trailing markdown or text
            code = re.sub(r'\n```.*$', '', code)
            return code

        # Last resort: return cleaned response if it looks like code
        cleaned = response.strip()
        if any(kw in cleaned for kw in ['def ', 'import ', 'print(', 'return ']):
            return cleaned

        return "⚠️ Could not extract valid Python code from the response"

    except Exception as e:
        return f"⚠️ Error extracting code: {str(e)}"

def generate_code(prompt):
    """Generate code using the model"""
    try:
        model, tokenizer = load_model()

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=600,
                temperature=0.1,
                top_p=0.95,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Remove the input prompt from output
        if prompt in generated_text:
            generated_text = generated_text.replace(prompt, "")

        return generated_text.strip()

    except Exception as e:
        return f"❌ Generation Error: {str(e)}"

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/generate", methods=["GET", "POST"])
def generate():
    code_output = ""
    error_msg = ""

    if request.method == "POST":
        prompt_type = request.form.get("prompt_type")

        try:
            if prompt_type == "file":
                f = request.files.get("code_file")
                if not f or f.filename == "":
                    flash("⚠️ No file uploaded!", "warning")
                    return redirect(url_for("generate"))

                filepath = os.path.join(app.config['UPLOAD_FOLDER'], f.filename)
                f.save(filepath)

                with open(filepath, "r", encoding="utf8") as infile:
                    file_code = infile.read()

                task = f"Improve and fix this Python code:\n\n{file_code}"
            else:
                user_prompt = request.form.get("user_prompt", "").strip()
                if not user_prompt:
                    flash("⚠️ Please enter a description", "warning")
                    return redirect(url_for("generate"))
                task = user_prompt

            prompt = prompt_code_generation(task)
            result = generate_code(prompt)
            code_output = extract_code(result)

        except Exception as e:
            error_msg = f"❌ Error: {str(e)}"

    return render_template("generate.html", code_output=code_output, error_msg=error_msg)

@app.route("/check", methods=["GET", "POST"])
def check():
    vuln_output = ""
    code_input = ""
    error_msg = ""

    if request.method == "POST":
        prompt_type = request.form.get("prompt_type")

        try:
            if prompt_type == "file":
                f = request.files.get("code_file")
                if not f or f.filename == "":
                    flash("⚠️ No file uploaded!", "warning")
                    return redirect(url_for("check"))

                filepath = os.path.join(app.config['UPLOAD_FOLDER'], f.filename)
                f.save(filepath)

                with open(filepath, "r", encoding="utf8") as infile:
                    code_input = infile.read()
            else:
                code_input = request.form.get("user_prompt", "").strip()
                if not code_input:
                    flash("⚠️ Please paste your code", "warning")
                    return redirect(url_for("check"))

            prompt = prompt_vuln_check(code_input)
            result = generate_code(prompt)

            # Clean up the vulnerability report
            vuln_output = re.sub(r'\[INST\].*?\[/INST\]', '', result, flags=re.DOTALL).strip()

        except Exception as e:
            error_msg = f"❌ Error: {str(e)}"

    return render_template("check.html", vuln_output=vuln_output, code_input=code_input, error_msg=error_msg)

if __name__ == "__main__":
    print("🚀 Starting SecureCodeGen AI...")
    print("⏳ Loading model (this may take 2-3 minutes)...")
    load_model()  # Preload model
    print("✅ Server ready!")
    app.run(host="0.0.0.0", port=8000, debug=False)

Overwriting app.py


📘 8️⃣ UI Templates Overview

The application includes 3 templates:

### 1. index.html
- Landing page
- Two feature cards:
  - Code Generation
  - Security Audit
- Animated hero banner

### 2. generate.html
- Prompt type selector (Text / File)
- Large code textarea
- File upload support
- Copy-to-clipboard code block
- Animated results panel

### 3. check.html
- Paste / Upload code
- Detailed AI-generated security report
- Clean formatting for readability

All templates use the same dark, professional UI theme.


In [ ]:
# ===============================
# 5️⃣ HTML Templates
# ===============================
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SecureCodeGen AI</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="particles"></div>
    <div class="container">
        <div class="hero-section fade-in">
            <div class="logo-container">
                <span class="logo-icon">🛡️</span>
                <h1 class="main-title">SecureCodeGen AI</h1>
            </div>
            <p class="hero-subtitle">AI-Powered Code Generation & Security Analysis</p>
            <p class="hero-description">CodeLlama-7B • 4-bit Quantized • Production Ready</p>
        </div>

        <div class="features-grid">
            <a href="{{ url_for('generate') }}" class="feature-card slide-up" style="animation-delay: 0.1s">
                <div class="card-icon">⚡</div>
                <h2>Code Generation</h2>
                <p>Generate production-ready Python code from natural language</p>
                <div class="card-arrow">→</div>
            </a>

            <a href="{{ url_for('check') }}" class="feature-card slide-up" style="animation-delay: 0.2s">
                <div class="card-icon">🔍</div>
                <h2>Security Audit</h2>
                <p>Deep vulnerability analysis and security recommendations</p>
                <div class="card-arrow">→</div>
            </a>
        </div>

        <div class="stats-section slide-up" style="animation-delay: 0.3s">
            <div class="stat-item">
                <div class="stat-number">7B</div>
                <div class="stat-label">Parameters</div>
            </div>
            <div class="stat-item">
                <div class="stat-number">T4</div>
                <div class="stat-label">GPU</div>
            </div>
            <div class="stat-item">
                <div class="stat-number">4-bit</div>
                <div class="stat-label">Quantized</div>
            </div>
        </div>

        <footer class="footer slide-up" style="animation-delay: 0.4s">
            <p>🔬 Powered by Meta's CodeLlama-7B-Instruct</p>
        </footer>
    </div>
</body>
</html>

Overwriting templates/index.html


In [ ]:
%%writefile templates/generate.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Code Generation</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="particles"></div>
    <div class="container">
        <div class="page-header fade-in">
            <a href="{{ url_for('index') }}" class="back-button">← Back</a>
            <h1 class="page-title">⚡ Code Generation</h1>
            <p class="page-subtitle">Describe what you want to build</p>
        </div>

        {% with messages = get_flashed_messages(with_categories=true) %}
            {% if messages %}
                {% for category, message in messages %}
                <div class="alert alert-{{ category }} slide-up">{{ message }}</div>
                {% endfor %}
            {% endif %}
        {% endwith %}

        <form method="POST" enctype="multipart/form-data" class="main-form slide-up">
            <div class="input-method-selector">
                <label class="radio-option">
                    <input type="radio" name="prompt_type" value="text" checked onchange="switchInput()">
                    <span class="radio-label">
                        <span class="radio-icon">✍️</span>
                        Text Description
                    </span>
                </label>
                <label class="radio-option">
                    <input type="radio" name="prompt_type" value="file" onchange="switchInput()">
                    <span class="radio-label">
                        <span class="radio-icon">📁</span>
                        Upload File
                    </span>
                </label>
            </div>

            <div id="text_prompt" class="input-section">
                <label class="input-label">What do you want to create?</label>
                <textarea
                    name="user_prompt"
                    rows="8"
                    placeholder="Example: Write a function to check if a number is an Armstrong number, with a main() that prints all Armstrong numbers from 1-1000"
                    class="code-textarea"
                ></textarea>
            </div>

            <div id="file_prompt" class="input-section" style="display:none;">
                <label class="input-label">Upload Python File</label>
                <input type="file" name="code_file" accept=".py,.txt" class="file-input">
            </div>

            <button type="submit" class="submit-button">
                <span class="button-icon">🚀</span>
                Generate Code
            </button>
        </form>

        {% if error_msg %}
        <div class="alert alert-error slide-up">{{ error_msg }}</div>
        {% endif %}

        {% if code_output %}
        <div class="output-section slide-up">
            <div class="output-header">
                <h3>✨ Generated Code</h3>
                <button onclick="copyCode()" class="copy-button" id="copy-btn">
                    <span>📋 Copy</span>
                </button>
            </div>
            <pre class="code-output"><code id="code-content">{{ code_output }}</code></pre>
        </div>
        {% endif %}
    </div>

    <script>
        function switchInput() {
            const selected = document.querySelector('input[name="prompt_type"]:checked').value;
            document.getElementById('text_prompt').style.display = selected === 'text' ? 'block' : 'none';
            document.getElementById('file_prompt').style.display = selected === 'file' ? 'block' : 'none';
        }

        function copyCode() {
            const code = document.getElementById('code-content').textContent;
            navigator.clipboard.writeText(code).then(() => {
                const btn = document.getElementById('copy-btn');
                btn.innerHTML = '<span>✅ Copied!</span>';
                setTimeout(() => {
                    btn.innerHTML = '<span>📋 Copy</span>';
                }, 2000);
            });
        }
    </script>
</body>
</html>

Overwriting templates/generate.html


In [ ]:
%%writefile templates/check.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Security Audit</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="particles"></div>
    <div class="container">
        <div class="page-header fade-in">
            <a href="{{ url_for('index') }}" class="back-button">← Back</a>
            <h1 class="page-title">🔍 Security Audit</h1>
            <p class="page-subtitle">Analyze code for vulnerabilities</p>
        </div>

        {% with messages = get_flashed_messages(with_categories=true) %}
            {% if messages %}
                {% for category, message in messages %}
                <div class="alert alert-{{ category }} slide-up">{{ message }}</div>
                {% endfor %}
            {% endif %}
        {% endwith %}

        <form method="POST" enctype="multipart/form-data" class="main-form slide-up">
            <div class="input-method-selector">
                <label class="radio-option">
                    <input type="radio" name="prompt_type" value="text" checked onchange="switchInput()">
                    <span class="radio-label">
                        <span class="radio-icon">✍️</span>
                        Paste Code
                    </span>
                </label>
                <label class="radio-option">
                    <input type="radio" name="prompt_type" value="file" onchange="switchInput()">
                    <span class="radio-label">
                        <span class="radio-icon">📁</span>
                        Upload File
                    </span>
                </label>
            </div>

            <div id="text_prompt" class="input-section">
                <label class="input-label">Python Code to Analyze</label>
                <textarea
                    name="user_prompt"
                    rows="12"
                    placeholder="Paste your Python code here..."
                    class="code-textarea"
                ></textarea>
            </div>

            <div id="file_prompt" class="input-section" style="display:none;">
                <label class="input-label">Upload Python File</label>
                <input type="file" name="code_file" accept=".py,.txt" class="file-input">
            </div>

            <button type="submit" class="submit-button">
                <span class="button-icon">🔍</span>
                Analyze Security
            </button>
        </form>

        {% if error_msg %}
        <div class="alert alert-error slide-up">{{ error_msg }}</div>
        {% endif %}

        {% if vuln_output %}
        <div class="output-section slide-up">
            <div class="output-header">
                <h3>📊 Security Report</h3>
            </div>
            <div class="security-report">
                <pre>{{ vuln_output }}</pre>
            </div>
        </div>
        {% endif %}
    </div>

    <script>
        function switchInput() {
            const selected = document.querySelector('input[name="prompt_type"]:checked').value;
            document.getElementById('text_prompt').style.display = selected === 'text' ? 'block' : 'none';
            document.getElementById('file_prompt').style.display = selected === 'file' ? 'block' : 'none';
        }
    </script>
</body>
</html>

Overwriting templates/check.html


📘 9️⃣ CSS Overview

The CSS file includes:

- Custom CSS variables for theme colors
- Modern dark UI inspired by VS Code
- Floating particle background
- Responsive grid layout
- Hover transitions & animations
- Styled buttons, textareas, file inputs
- Clean code block rendering
- Fade-in, slide-up & pulse animations

Supports desktop + mobile responsiveness.


In [ ]:
# ===============================
# 6️⃣ Complete Professional CSS
# ===============================
%%writefile static/style.css
:root {
    --primary: #6366f1;
    --primary-dark: #4f46e5;
    --primary-light: #818cf8;
    --secondary: #10b981;
    --danger: #ef4444;
    --warning: #f59e0b;
    --dark-bg: #0f172a;
    --card-bg: #1e293b;
    --card-hover: #334155;
    --text-primary: #f1f5f9;
    --text-secondary: #cbd5e1;
    --border: #475569;
}

* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

body {
    font-family: 'Inter', system-ui, -apple-system, sans-serif;
    background: linear-gradient(135deg, var(--dark-bg) 0%, #1a1f35 100%);
    color: var(--text-primary);
    min-height: 100vh;
    position: relative;
}

.particles {
    position: fixed;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    pointer-events: none;
    background-image:
        radial-gradient(circle at 20% 30%, rgba(99, 102, 241, 0.1) 0%, transparent 50%),
        radial-gradient(circle at 80% 70%, rgba(16, 185, 129, 0.08) 0%, transparent 50%);
    animation: float 20s ease-in-out infinite;
}

@keyframes float {
    0%, 100% { transform: translateY(0px); }
    50% { transform: translateY(-20px); }
}

@keyframes fadeIn {
    from { opacity: 0; transform: translateY(20px); }
    to { opacity: 1; transform: translateY(0); }
}

@keyframes slideUp {
    from { opacity: 0; transform: translateY(30px); }
    to { opacity: 1; transform: translateY(0); }
}

@keyframes pulse {
    0%, 100% { transform: scale(1); }
    50% { transform: scale(1.05); }
}

.fade-in {
    animation: fadeIn 0.6s ease-out;
}

.slide-up {
    animation: slideUp 0.6s ease-out forwards;
    opacity: 0;
}

.container {
    max-width: 1200px;
    margin: 0 auto;
    padding: 40px 20px;
    position: relative;
    z-index: 1;
}

/* Hero Section */
.hero-section {
    text-align: center;
    margin-bottom: 60px;
}

.logo-container {
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 15px;
    margin-bottom: 20px;
}

.logo-icon {
    font-size: 4em;
    filter: drop-shadow(0 0 20px rgba(99, 102, 241, 0.5));
    animation: pulse 3s ease-in-out infinite;
}

.main-title {
    font-size: 3.5em;
    font-weight: 800;
    background: linear-gradient(135deg, var(--primary-light), var(--secondary));
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;
}

.hero-subtitle {
    font-size: 1.3em;
    color: var(--text-secondary);
    margin: 15px 0;
}

.hero-description {
    font-size: 0.95em;
    color: var(--text-secondary);
    opacity: 0.7;
}

/* Features Grid */
.features-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
    gap: 30px;
    margin-bottom: 50px;
}

.feature-card {
    background: var(--card-bg);
    border: 1px solid var(--border);
    border-radius: 20px;
    padding: 40px 30px;
    text-decoration: none;
    color: var(--text-primary);
    transition: all 0.3s ease;
    position: relative;
    overflow: hidden;
}

.feature-card::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    background: linear-gradient(135deg, rgba(99, 102, 241, 0.1), transparent);
    opacity: 0;
    transition: opacity 0.3s;
}

.feature-card:hover {
    transform: translateY(-10px);
    box-shadow: 0 20px 40px rgba(0,0,0,0.3);
    border-color: var(--primary);
}

.feature-card:hover::before {
    opacity: 1;
}

.card-icon {
    font-size: 3em;
    margin-bottom: 20px;
}

.feature-card h2 {
    font-size: 1.8em;
    margin-bottom: 15px;
}

.feature-card p {
    color: var(--text-secondary);
    line-height: 1.6;
}

.card-arrow {
    position: absolute;
    bottom: 30px;
    right: 30px;
    font-size: 2em;
    opacity: 0;
    transition: all 0.3s;
}

.feature-card:hover .card-arrow {
    opacity: 1;
    transform: translateX(5px);
}

/* Stats Section */
.stats-section {
    display: flex;
    justify-content: space-around;
    background: var(--card-bg);
    border-radius: 15px;
    padding: 30px;
    margin-bottom: 40px;
    border: 1px solid var(--border);
}

.stat-item {
    text-align: center;
}

.stat-number {
    font-size: 2.5em;
    font-weight: 800;
    background: linear-gradient(135deg, var(--primary), var(--secondary));
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;
}

.stat-label {
    color: var(--text-secondary);
    margin-top: 5px;
}

/* Page Header */
.page-header {
    text-align: center;
    margin-bottom: 50px;
}

.back-button {
    display: inline-block;
    color: var(--text-secondary);
    text-decoration: none;
    padding: 8px 16px;
    border-radius: 8px;
    margin-bottom: 20px;
    transition: all 0.3s;
}

.back-button:hover {
    background: var(--card-bg);
    color: var(--primary-light);
}

.page-title {
    font-size: 2.5em;
    font-weight: 800;
    margin-bottom: 10px;
}

.page-subtitle {
    color: var(--text-secondary);
    font-size: 1.1em;
}

/* Forms */
.main-form {
    background: var(--card-bg);
    border: 1px solid var(--border);
    border-radius: 20px;
    padding: 40px;
    margin-bottom: 30px;
}

.input-method-selector {
    display: flex;
    gap: 20px;
    margin-bottom: 30px;
}

.radio-option {
    flex: 1;
    cursor: pointer;
}

.radio-option input[type="radio"] {
    display: none;
}

.radio-label {
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;
    padding: 15px;
    background: var(--dark-bg);
    border: 2px solid var(--border);
    border-radius: 12px;
    transition: all 0.3s;
}

.radio-icon {
    font-size: 1.5em;
}

.radio-option input[type="radio"]:checked + .radio-label {
    background: linear-gradient(135deg, var(--primary), var(--primary-dark));
    border-color: var(--primary);
}

.input-section {
    margin-bottom: 25px;
}

.input-label {
    display: block;
    margin-bottom: 12px;
    color: var(--text-secondary);
    font-weight: 600;
    text-transform: uppercase;
    font-size: 0.9em;
    letter-spacing: 0.5px;
}

.code-textarea {
    width: 100%;
    padding: 15px;
    background: var(--dark-bg);
    border: 2px solid var(--border);
    border-radius: 12px;
    color: var(--text-primary);
    font-family: 'Fira Code', monospace;
    font-size: 0.95em;
    resize: vertical;
    transition: all 0.3s;
}

.code-textarea:focus {
    outline: none;
    border-color: var(--primary);
    box-shadow: 0 0 0 3px rgba(99, 102, 241, 0.1);
}

.file-input {
    width: 100%;
    padding: 12px;
    background: var(--dark-bg);
    border: 2px solid var(--border);
    border-radius: 12px;
    color: var(--text-primary);
    cursor: pointer;
    transition: all 0.3s;
}

.file-input:hover {
    border-color: var(--primary);
}

.submit-button {
    width: 100%;
    padding: 16px;
    background: linear-gradient(135deg, var(--primary), var(--primary-dark));
    color: white;
    border: none;
    border-radius: 12px;
    font-size: 1.1em;
    font-weight: 700;
    cursor: pointer;
    transition: all 0.3s;
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;
}

.submit-button:hover {
    transform: translateY(-2px);
    box-shadow: 0 10px 30px rgba(99, 102, 241, 0.4);
}

.button-icon {
    font-size: 1.2em;
}

/* Alerts */
.alert {
    padding: 15px 20px;
    border-radius: 12px;
    margin-bottom: 20px;
    font-weight: 500;
}

.alert-warning {
    background: rgba(245, 158, 11, 0.1);
    border: 1px solid var(--warning);
    color: var(--warning);
}

.alert-error {
    background: rgba(239, 68, 68, 0.1);
    border: 1px solid var(--danger);
    color: var(--danger);
}

/* Output Section */
.output-section {
    background: var(--card-bg);
    border: 1px solid var(--border);
    border-radius: 20px;
    padding: 30px;
}

.output-header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-bottom: 20px;
}

.output-header h3 {
    font-size: 1.5em;
    color: var(--primary-light);
}

.copy-button {
    padding: 10px 20px;
    background: var(--primary);
    color: white;
    border: none;
    border-radius: 8px;
    cursor: pointer;
    transition: all 0.3s;
    font-weight: 600;
}

.copy-button:hover {
    background: var(--primary-dark);
    transform: translateY(-2px);
}

.code-output {
    background: var(--dark-bg);
    padding: 20px;
    border-radius: 12px;
    overflow-x: auto;
    margin: 0;
}

.code-output code {
    font-family: 'Fira Code', 'Courier New', monospace;
    color: #e0e0e0;
    font-size: 0.95em;
    line-height: 1.6;
}

.security-report {
    background: var(--dark-bg);
    padding: 20px;
    border-radius: 12px;
}

.security-report pre {
    color: var(--text-primary);
    font-family: 'Fira Code', monospace;
    font-size: 0.95em;
    line-height: 1.8;
    margin: 0;
    white-space: pre-wrap;
}

/* Footer */
.footer {
    text-align: center;
    padding: 20px;
    color: var(--text-secondary);
    font-size: 0.9em;
}

/* Responsive */
@media (max-width: 768px) {
    .main-title {
        font-size: 2.5em;
    }

    .features-grid {
        grid-template-columns: 1fr;
    }

    .stats-section {
        flex-direction: column;
        gap: 20px;
    }

    .main-form {
        padding: 20px;
    }

    .input-method-selector {
        flex-direction: column;
    }
}

Overwriting static/style.css


📘 🔟 Clean Startup (Kill Old Flask/ngrok)

Before starting the server:

!pkill -f flask
!pkill -f ngrok

This prevents:
- Port 8000 conflicts
- Duplicate model loads
- ngrok "tunnel session failed" errors


In [ ]:
# Kill any previous processes
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"


^C
^C


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 5835


📘 1️⃣1️⃣ Run Flask Server

Run in background:

!nohup python app.py > flask.log 2>&1 &

All logs are stored in:
flask.log


In [ ]:
# Run Flask in the background
!nohup python app.py > flask.log 2>&1 &


📘 1️⃣2️⃣ Start ngrok Tunnel (Token Required)

Generate token:
https://dashboard.ngrok.com/get-started/your-authtoken

Set it here:
conf.get_default().auth_token = "YOUR_NGROK_TOKEN"

Start the tunnel:
public_url = ngrok.connect(8000)

This gives you a public URL like:
https://abcd123.ngrok-free.app


In [ ]:
# Start ngrok tunnel
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# Optional: check logs
!sleep 3 && tail -n 20 flask.log


🌍 Public URL: NgrokTunnel: "https://68fe5233313b.ngrok-free.app" -> "http://localhost:8000"


📘 1️⃣3️⃣ Check Logs

Inspect backend logs:

!tail -n 20 flask.log
!tail -n 50 flask.log

Useful to debug:
- tokenizer issues
- quantization errors
- missing templates
- GPU memory issues


In [ ]:
!tail -n 50 flask.log

📘 1️⃣4️⃣ Notebook Completed

🎉 SecureCodeGen AI is ready to use!

Your application now supports:

✔ AI-powered Python code generation  
✔ Secure vulnerability auditing  
✔ Clean UI with animations  
✔ File upload & safe processing  
✔ 4-bit quantized LLM backend  
✔ Public access via ngrok  

You can deploy this on:
- GitHub
- Personal portfolio
- Internal company tools
- Classroom demonstrations  


📘 1️⃣5️⃣ Learning Resources (Official Websites)

# AI & Security Resources

## CodeLlama Official Models
https://huggingface.co/codellama

## HuggingFace Transformers Documentation
https://huggingface.co/docs/transformers

## BitsAndBytes (4-bit Quantization)
https://github.com/TimDettmers/bitsandbytes

## Accelerate Library (Hardware Optimization)
https://huggingface.co/docs/accelerate

## Flask (Official Website)
https://flask.palletsprojects.com/

## Pyngrok Documentation
https://pyngrok.readthedocs.io/

## OWASP Python Security Guidelines
https://cheatsheetseries.owasp.org/

## Static Code Analysis Concepts (SAST)
https://owasp.org/www-community/controls/Static_Code_Analysis
